In [17]:
# Dependencies 
import pandas as pd
import os
from datetime import datetime
from bs4 import BeautifulSoup
import glob as glob

In [18]:
# Import alternate championhsip points tables
points_table_formula1 = pd.read_csv('points_systems_csv/2024_formula1_race_points_table.csv')
points_table_imsa = pd.read_csv('points_systems_csv/2024_imsa_race_points_table.csv')
points_table_imsa_qual = pd.read_csv('points_systems_csv/2024_imsa_qualifying_points_table.csv')


In [19]:
all_files_path = 'race_results_html/*.htm'
race_results_files = []
race_results_files = glob.glob(all_files_path)

season_races_df = pd.DataFrame()

# Initialize a list to contain the race results 
race_result_data = []
i = 0

for race_result_file in race_results_files:
# Get City / Race Name
    i += 1
    j = 0
    city = race_result_file.split('.')[0]
    city = city[18:]
# Read html file and store as html_content
    with open(race_result_file, 'r') as file:
        html_content = file.read()
    # Parse race result table html
    soup = BeautifulSoup(html_content, 'html.parser')
    race_table_html = soup.find(id = 'race-season')

# Loop through table html and parse to list
    for row in race_table_html.find_all('tr'):
        j += 1
        row_data = [cell.get_text(strip=True) for cell in row.find_all(['th', 'td'])]
        if j == 1 and i == 1:
            row_data_with_city = row_data + ['City']
            race_result_data.append(row_data_with_city)
        elif j != 1:
            row_data_with_city = row_data + [city]
            race_result_data.append(row_data_with_city)
        else:
            pass

    race_result_df = pd.DataFrame(race_result_data[1:], columns=race_result_data[0])
    race_result_df['Rank'] = race_result_df['Rank'].astype('int')
    race_result_df['Starts'] = race_result_df['Starts'].astype('int')
    race_result_df['Laps'] = race_result_df['Laps'].astype('int')
    race_result_df['Laps Led'] = race_result_df['Laps Led'].replace('-', 0).astype('int')
    race_result_df['Points'] = race_result_df['Points'].astype('int')
    race_result_df['Pit stop'] = race_result_df['Pit stop'].astype('int64')
    race_result_df['Total time'] = pd.to_timedelta(race_result_df['Total time'])
    race_result_df['Total time in seconds'] = race_result_df['Total time'].dt.total_seconds()

    race_result_df = pd.merge(race_result_df, points_table_formula1, on='Rank', how='inner')
    race_result_df = pd.merge(race_result_df, points_table_imsa, on='Rank', how='inner')
    race_result_df = pd.merge(race_result_df, points_table_imsa_qual, on='Starts', how='inner')
    race_result_df['Points (IMSA Scoring)'] = race_result_df['Points (IMSA Scoring)']+race_result_df['Points (IMSA Scoring) - Qualifying']
    race_result_df = race_result_df.drop(columns=['Points (IMSA Scoring) - Qualifying'])

    

race_result_df.head()

,Rank,Driver,Car No.,Starts,Laps,Total time,Laps Led,Status,Points,Avg.Speed,Pit stop,City,Total time in seconds,Points (F1 Scoring),Points (IMSA Scoring)
0,1,Scott Dixon,9,5,100,0 days 02:06:07.968400,35,Running,53,78.251,2,detroit,7567.9684,25,376
1,2,Marcus Ericsson,28,9,100,0 days 02:06:08.825100,0,Running,40,78.242,2,detroit,7568.8251,18,342
2,3,Marcus Armstrong,11,19,100,0 days 02:06:12.881300,0,Running,35,78.200,4,detroit,7572.8813,15,312
3,4,Kyle Kirkwood,27,6,100,0 days 02:06:14.093300,24,Running,33,78.188,2,detroit,7574.0933,12,305
4,5,Alexander Rossi,7,16,100,0 days 02:06:16.921600,0,Running,30,78.158,4,detroit,7576.9216,10,275


In [20]:
# Get a list of cities from race_result_df and convert it to a data frame
race_numbers = race_result_df.groupby('City')['City'].count()
race_numbers_df = pd.DataFrame(race_numbers)

# Define each race's race number
race_numbers_df.at['stpetersburg', 'Race Number'] = 1
race_numbers_df.at['longbeach', 'Race Number'] = 2
race_numbers_df.at['barber', 'Race Number'] = 3
race_numbers_df.at['indyrc', 'Race Number'] = 4
race_numbers_df.at['indy500', 'Race Number'] = 5
race_numbers_df.at['detroit', 'Race Number'] = 6
race_numbers_df.at['roadamerica', 'Race Number'] = 7
race_numbers_df.at['lagunaseca', 'Race Number'] = 8
race_numbers_df.at['midohio', 'Race Number'] = 9
race_numbers_df.at['iowa1', 'Race Number'] = 10
race_numbers_df.at['iowa2', 'Race Number'] = 11
race_numbers_df.at['toronto', 'Race Number'] = 12
race_numbers_df.at['gateway', 'Race Number'] = 13
race_numbers_df.at['portland', 'Race Number'] = 14
race_numbers_df.at['milwaukee1', 'Race Number'] = 15
race_numbers_df.at['milwaukee2', 'Race Number'] = 16
race_numbers_df.at['nashville', 'Race Number'] = 17

# Define track types for each race
race_numbers_df.at['stpetersburg', 'Track Type'] = 'Street Course'
race_numbers_df.at['longbeach', 'Track Type'] = 'Street Course'
race_numbers_df.at['barber', 'Track Type'] = 'Road Course'
race_numbers_df.at['indyrc', 'Track Type'] = 'Road Course'
race_numbers_df.at['indy500', 'Track Type'] = 'Oval'
race_numbers_df.at['detroit', 'Track Type'] = 'Street Course'
race_numbers_df.at['roadamerica', 'Track Type'] = 'Road Course'
race_numbers_df.at['lagunaseca', 'Track Type'] = 'Road Course'
race_numbers_df.at['midohio', 'Track Type'] = 'Road Course'
race_numbers_df.at['iowa1', 'Track Type'] = 'Oval'
race_numbers_df.at['iowa2', 'Track Type'] = 'Oval'
race_numbers_df.at['toronto', 'Track Type'] = 'Street Course'
race_numbers_df.at['gateway', 'Track Type'] = 'Oval'
race_numbers_df.at['portland', 'Track Type'] = 'Road Course'
race_numbers_df.at['milwaukee1', 'Track Type'] = 'Oval'
race_numbers_df.at['milwaukee2', 'Track Type'] = 'Oval'
race_numbers_df.at['nashville', 'Track Type'] = 'Oval'

#Clean the data frame
race_numbers_df['Race Number'] = race_numbers_df['Race Number'].astype('int64')
race_numbers_df = race_numbers_df.sort_values(by='Race Number')
race_numbers_df.rename(columns={'City': 'City2'}, inplace=True)
race_numbers_df.drop(columns=['City2'], inplace=True)
race_numbers_df = race_numbers_df.reset_index()

# Merge the data frames, thereby giving the race_result_df dataframe a race number 
race_result_df = pd.merge(race_result_df, race_numbers_df, on='City', how='inner')
race_result_df


,Rank,Driver,Car No.,Starts,Laps,Total time,Laps Led,Status,Points,Avg.Speed,Pit stop,City,Total time in seconds,Points (F1 Scoring),Points (IMSA Scoring),Race Number,Track Type
0,1,Scott Dixon,9,5,100,0 days 02:06:07.968400,35,Running,53,78.251,2,detroit,7567.9684,25,376,6,Street Course
1,2,Marcus Ericsson,28,9,100,0 days 02:06:08.825100,0,Running,40,78.242,2,detroit,7568.8251,18,342,6,Street Course
2,3,Marcus Armstrong,11,19,100,0 days 02:06:12.881300,0,Running,35,78.200,4,detroit,7572.8813,15,312,6,Street Course
3,4,Kyle Kirkwood,27,6,100,0 days 02:06:14.093300,24,Running,33,78.188,2,detroit,7574.0933,12,305,6,Street Course
4,5,Alexander Rossi,7,16,100,0 days 02:06:16.921600,0,Running,30,78.158,4,detroit,7576.9216,10,275,6,Street Course
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,23,Kyffin Simpson,4,22,86,0 days 01:50:32.231200,0,Contact,7,104.472,3,lagunaseca,6632.2312,0,89,8,Road Course
462,24,Graham Rahal,15,19,86,0 days 01:50:33.260000,0,Contact,6,104.456,4,lagunaseca,6633.2600,0,82,8,Road Course
463,25,Jack Harvey,18,26,82,0 days 01:44:40.785300,0,Mechanical,5,105.187,3,lagunaseca,6280.7853,0,65,8,Road Course
464,26,Rinus VeeKay,21,20,72,0 days 01:39:43.039800,0,Mechanical,5,96.956,3,lagunaseca,5983.0398,0,61,8,Road Course


In [21]:
print(race_result_df.dtypes)

race_result_df.drop(columns = 'Total time', inplace=True)

race_result_df.rename(columns = {
    "Race Number": "race_num",
    "City": "race_city",
    "Rank": "rank",
    "Driver": "driver",
    "Car No.": "car_no",
    "Starts": "start",
    "Laps": "laps",
    "Total time in seconds": "total_time",
    "Laps Led": "laps_led",
    "Status": "status",
    "Avg.Speed": "avg_speed",
    "Pit stop": "num_pit_stop",
    "Points": "points",
    "Points (F1 Scoring)": "points_f1",
    "Points (IMSA Scoring)": "points_IMSA", 
    "Track Type" : "track_type"
}, inplace=True)

rev_order = ["race_num", "race_city", "rank", "driver", "car_no", "start", "laps", "total_time", "laps_led", "status", "avg_speed", "num_pit_stop", "points", "points_f1", "points_IMSA", "track_type"]
reordered_race_result_df = race_result_df[rev_order]

reordered_race_result_df.to_csv('Resources/indydata_2024.csv', index=False, header=True)

Rank                               int64
Driver                            object
Car No.                           object
Starts                             int64
Laps                               int64
Total time               timedelta64[ns]
Laps Led                           int64
Status                            object
Points                             int64
Avg.Speed                         object
Pit stop                           int64
City                              object
Total time in seconds            float64
Points (F1 Scoring)                int64
Points (IMSA Scoring)              int64
Race Number                        int64
Track Type                        object
dtype: object
